In [6]:
# import libraries
import os
import pandas as pd
import json
import gzip
import fasttext 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import re
import numpy as np
import sklearn

In [2]:
small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/CSV/medium_tables_seq.csv')

In [3]:
small.drop(small[(small['name']=='inalbum') &(small['class']=='MusicRecording')].index, inplace=True)
small.drop(small[(small['name']=='performer') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventattendancemode') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventstatus') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='owns') &(small['class']=='LocalBusiness')].index, inplace=True)
small["label_complete"] = small["name"] + small["class"]
num_columns = len(set.union(set(small.label_complete.unique())))
small

,Unnamed: 0,text,name,class,label_complete
0,0,Heavenly Sentiments Spray Peaceful White Lilie...,name,Product,nameProduct
1,1,"{'pricecurrency': 'USD', 'lowprice': '202.95',...",offers,Product,offersProduct
2,2,"{'reviewcount': '4', 'ratingvalue': '5.00'},{'...",aggregaterating,Product,aggregateratingProduct
3,3,"[{'author': 'from -', 'description': 'Arrangem...",review,Product,reviewProduct
4,4,NewCondition NewCondition NewCondition NewCond...,itemcondition,Product,itemconditionProduct
...,...,...,...,...,...
21452,21452,5.0 cu. ft. Smart Front Load Washer with Load ...,name,Product,nameProduct
21453,21453,"{'pricecurrency': 'USD', 'price': '1349.99', '...",offers,Product,offersProduct
21454,21454,"{'name': 'Whirlpool'},{'name': 'GE Appliances'...",brand,Product,brandProduct
21455,21455,"{'reviewcount': '7', 'itemreviewed': '5.0 cu. ...",aggregaterating,Product,aggregateratingProduct


In [5]:
test = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/test_tables_seq_9000.csv')
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='owns') &(test['class']=='LocalBusiness')].index, inplace=True)
test["label_complete"] = test["name"] + test["class"]
num_columns = len(set.union(set(test.label_complete.unique())))
test

,Unnamed: 0,text,name,class,label_complete
0,0,Plauen | Indonesien Freiburg | Iraq-Kurdistan ...,name,Event,nameEvent
1,1,2022-03-27T16:30:00+01:00 2020-11-15T22:00:00+...,enddate,Event,enddateEvent
2,2,"{'name': 'Abenteuer Arena'},{'name': 'Paulussa...",location,Event,locationEvent
3,3,"{'name': 'Freiträumer Festival'},{'name': 'MUN...",organizer,Event,organizerEvent
4,4,Máy làm kem Sunny Way RB1119A TỦ MÁT 2 CÁNH KÍ...,name,Product,nameProduct
...,...,...,...,...,...
48530,48530,"{'name': 'Virtually'},{'name': 'Virtually'},{'...",location,Event,locationEvent
48531,48531,{'email': 'c&#104;i&#108;&#100;&#114;&#101;n&#...,organizer,Event,organizerEvent
48532,48532,Marina Olivier Celia Tatjana Valentine Gaia Ré...,givenname,Person,givennamePerson
48533,48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,familyname,Person,familynamePerson


In [6]:
test.drop(['Unnamed: 0','class'], axis=1, inplace=True)
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [7]:
small.drop(['Unnamed: 0','class'], axis=1, inplace=True)
small['text'] = small.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [21]:
labels_new = {'worstratingProduct': 0,
 'acceptsreservationsRestaurant': 1,
 'additionalnamePerson': 2,
 'additionalpropertyProduct': 3,
 'addressHotel': 4,
 'addressLocalBusiness': 5,
 'addressPerson': 6,
 'addressProduct': 7,
 'addressRestaurant': 8,
 'addresslocalityLocalBusiness': 9,
 'addressregionLocalBusiness': 10,
 'affiliationPerson': 11,
 'aggregateratingBook': 12,
 'aggregateratingCreativeWork': 13,
 'aggregateratingHotel': 14,
 'aggregateratingLocalBusiness': 15,
 'aggregateratingProduct': 16,
 'aggregateratingRecipe': 17,
 'aggregateratingRestaurant': 18,
 'alternatenameProduct': 19,
 'alternativeheadlineCreativeWork': 20,
 'areaservedLocalBusiness': 21,
 'articlebodyCreativeWork': 22,
 'audienceProduct': 23,
 'authorBook': 24,
 'authorCreativeWork': 25,
 'authorProduct': 26,
 'authorRecipe': 27,
 'availabilityProduct': 28,
 'bestratingProduct': 29,
 'birthdatePerson': 30,
 'birthplacePerson': 31,
 'bookeditionBook': 32,
 'bookformatBook': 33,
 'brandProduct': 34,
 'breadcrumbProduct': 35,
 'byartistMusicAlbum': 36,
 'byartistMusicRecording': 37,
 'categoryProduct': 38,
 'citystatezipLocalBusiness': 39,
 'colorProduct': 40,
 'commentcountCreativeWork': 41,
 'conditionProduct': 42,
 'contactpointLocalBusiness': 43,
 'contactpointPerson': 44,
 'cookingmethodRecipe': 45,
 'cooktimeRecipe': 46,
 'copyrightholderCreativeWork': 47,
 'copyrightyearCreativeWork': 48,
 'creatorCreativeWork': 49,
 'datecreatedCreativeWork': 50,
 'datecreatedMusicRecording': 51,
 'datemodifiedCreativeWork': 52,
 'datemodifiedRecipe': 53,
 'datepublishedBook': 54,
 'datepublishedCreativeWork': 55,
 'datepublishedMusicRecording': 56,
 'datepublishedProduct': 57,
 'datepublishedRecipe': 58,
 'deathdatePerson': 59,
 'depthProduct': 60,
 'disambiguatingdescriptionProduct': 61,
 'doortimeEvent': 62,
 'durationEvent': 63,
 'durationMusicRecording': 64,
 'emailHotel': 65,
 'emailLocalBusiness': 66,
 'emailPerson': 67,
 'enddateEvent': 68,
 'episodenumberTVEpisode': 69,
 'worksforPerson': 70,
 'familynamePerson': 71,
 'faxnumberLocalBusiness': 72,
 'faxnumberPerson': 73,
 'genderPerson': 74,
 'genreBook': 75,
 'genreCreativeWork': 76,
 'genreMusicRecording': 77,
 'geoHotel': 78,
 'geoLocalBusiness': 79,
 'geoPlace': 80,
 'geoProduct': 81,
 'geoRestaurant': 82,
 'givennamePerson': 83,
 'gtin12Product': 84,
 'gtin13Product': 85,
 'gtin14Product': 86,
 'gtin8Product': 87,
 'gtinProduct': 88,
 'hasmapLocalBusiness': 89,
 'hasmenuRestaurant': 90,
 'headlineCreativeWork': 91,
 'headlineRecipe': 92,
 'heightPerson': 93,
 'heightProduct': 94,
 'homelocationPerson': 95,
 'identifierProduct': 96,
 'worklocationPerson': 97,
 'ingredientsRecipe': 98,
 'inlanguageBook': 99,
 'inlanguageCreativeWork': 100,
 'interactioncountCreativeWork': 101,
 'interactionstatisticCreativeWork': 102,
 'interactiontypeCreativeWork': 103,
 'isbnBook': 104,
 'ispartofRecipe': 105,
 'isrelatedtoProduct': 106,
 'issimilartoProduct': 107,
 'itemconditionProduct': 108,
 'itemlistelementProduct': 109,
 'jobtitlePerson': 110,
 'keywordsCreativeWork': 111,
 'keywordsRecipe': 112,
 'knowslanguagePerson': 113,
 'legalnameLocalBusiness': 114,
 'locationEvent': 115,
 'locationLocalBusiness': 116,
 'mainentityofpageCreativeWork': 117,
 'weightProduct': 118,
 'mainentityofpageProduct': 119,
 'mainentityofpageRecipe': 120,
 'makesofferPerson': 121,
 'manufacturerProduct': 122,
 'materialProduct': 123,
 'memberofPerson': 124,
 'menuRestaurant': 125,
 'modelProduct': 126,
 'mpnProduct': 127,
 'nameBook': 128,
 'nameCreativeWork': 129,
 'nameEvent': 130,
 'nameHotel': 131,
 'nameLocalBusiness': 132,
 'nameMusicAlbum': 133,
 'nameMusicRecording': 134,
 'namePlace': 135,
 'nameProduct': 136,
 'nameRecipe': 137,
 'nameRestaurant': 138,
 'nameTVEpisode': 139,
 'nationalityPerson': 140,
 'numberofpagesBook': 141,
 'numtracksMusicAlbum': 142,
 'nutritionRecipe': 143,
 'weightPerson': 144,
 'offersBook': 145,
 'offersCreativeWork': 146,
 'offersProduct': 147,
 'openinghoursLocalBusiness': 148,
 'openinghoursRestaurant': 149,
 'openinghoursspecificationLocalBusiness': 150,
 'openinghoursspecificationPlace': 151,
 'openinghoursspecificationRestaurant': 152,
 'organizerEvent': 153,
 'partofseriesTVEpisode': 154,
 'paymentacceptedLocalBusiness': 155,
 'widthProduct': 156,
 'performersEvent': 157,
 'performtimeRecipe': 158,
 'postalcodeLocalBusiness': 159,
 'preptimeRecipe': 160,
 'priceProduct': 161,
 'pricecurrencyProduct': 162,
 'pricerangeHotel': 163,
 'pricerangeLocalBusiness': 164,
 'pricerangeRestaurant': 165,
 'publisherBook': 166,
 'publisherCreativeWork': 167,
 'publisherRecipe': 168,
 'ratingvalueProduct': 169,
 'recipecategoryRecipe': 170,
 'recipecuisineRecipe': 171,
 'recipeingredientRecipe': 172,
 'recipeinstructionsRecipe': 173,
 'recipeyieldRecipe': 174,
 'releasedateProduct': 175,
 'reviewLocalBusiness': 176,
 'reviewProduct': 177,
 'reviewRecipe': 178,
 'reviewcountProduct': 179,
 'reviewsProduct': 180,
 'sameasLocalBusiness': 181,
 'sameasPerson': 182,
 'sameasPlace': 183,
 'servescuisineRestaurant': 184,
 'shop-currencyProduct': 185,
 'starratingHotel': 186,
 'streetaddressLocalBusiness': 187,
 'suitablefordietRecipe': 188,
 'telephoneHotel': 189,
 'telephoneLocalBusiness': 190,
 'telephonePerson': 191,
 'telephonePlace': 192,
 'telephoneRestaurant': 193,
 'titleProduct': 194,
 'totaltimeRecipe': 195,
 'trackMusicAlbum': 196,
 'typicalagerangeEvent': 197,
 'versionCreativeWork': 198
 #'offerdetailsProduct': 144,
# 'founderLocalBusiness': 118,
 # 'mainentityofpagePerson': 118,
 #'eventattendancemodeEvent': 201,
 #'performerEvent': 202,
 #'inalbumMusicRecording': 203,
 #'eventstatusEvent': 204
 }

In [9]:
medium = small


In [10]:
test['label'] = test.label_complete.map(labels_new)

test.drop(['label_complete', 'name'], axis=1, inplace=True)
test

,text,label
0,Plauen Indonesien Freiburg Iraq-Kurdistan Boch...,130.0
1,2022-03-27T16 30 00 01 00 2020-11-15T22 00 00 ...,68.0
2,name Abenteuer Arena name Paulussaal name Deu...,115.0
3,name Freitr umer Festival name MUNDOlogia nam...,153.0
4,M y l m kem Sunny Way RB1119A T M T 2 C NH K N...,136.0
...,...,...
48530,name Virtually name Virtually name Virtually ...,115.0
48531,email c 104 i 108 100 114 101 n 115 st 97 f 1...,153.0
48532,Marina Olivier Celia Tatjana Valentine Gaia R ...,83.0
48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,71.0


In [11]:
medium['label'] = medium.label_complete.map(labels_new)

medium.drop(['label_complete', 'name'], axis=1, inplace=True)
medium

,text,label
0,Heavenly Sentiments Spray Peaceful White Lilie...,136.0
1,pricecurrency USD lowprice 202 95 highprice 2...,147.0
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,16.0
3,author from - description Arrangement was mad...,177.0
4,NewCondition NewCondition NewCondition NewCond...,108.0
...,...,...
21452,5 0 cu ft Smart Front Load Washer with Load Go...,136.0
21453,pricecurrency USD price 1349 99 availability ...,147.0
21454,name Whirlpool name GE Appliances name GE App...,34.0
21455,reviewcount 7 itemreviewed 5 0 cu ft Smart Fr...,16.0


In [12]:
test = test[~test['label'].isna()]

In [13]:
medium = medium[~medium['label'].isna()]

In [14]:
test['label'] = test.label.astype(int)

/tmp/ipykernel_1523150/2755059116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'] = test.label.astype(int)


In [15]:
medium['label'] = medium.label.astype(int)

/tmp/ipykernel_1523150/3777025901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medium['label'] = medium.label.astype(int)


In [16]:
len (test['label'].unique())

199

In [17]:
len (medium['label'].unique())

199

In [18]:
med_list = medium['label'].unique()

In [19]:
test_list = test['label'].unique()

In [20]:
set(test_list) - set(med_list)

set()

In [21]:
set(med_list) - set(test_list)

set()

In [23]:
sorted(medium.label.unique()) ==sorted(test.label.unique())

True

# Read in mid set 

In [25]:
test.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_tables_prepped.csv', index=False)

In [26]:
medium.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/medium_train_tables_prepped.csv', index=False)

In [2]:
test_med = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_tables_prepped.csv')

In [10]:
# 2 epochs

pred_med = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/predict_results_None.txt', sep='\t')

In [3]:
from sklearn.metrics import f1_score
import pandas as pd
from matplotlib.pyplot import figure

In [12]:
f1_score(pred_med.prediction, test_med.label, average='micro')

0.5623171083542843

In [3]:
pred_med = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/predict_results_None.txt', sep='\t')

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from matplotlib.pyplot import figure

In [5]:
f1_score(pred_med.prediction, test_med.label, average='micro')

0.7050653258047233

# Small

In [14]:
small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/CSV/small_tables_seq.csv')

In [15]:
small.drop(small[(small['name']=='inalbum') &(small['class']=='MusicRecording')].index, inplace=True)
small.drop(small[(small['name']=='performer') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventattendancemode') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='eventstatus') &(small['class']=='Event')].index, inplace=True)
small.drop(small[(small['name']=='owns') &(small['class']=='LocalBusiness')].index, inplace=True)
small["label_complete"] = small["name"] + small["class"]
num_columns = len(set.union(set(small.label_complete.unique())))
small

,Unnamed: 0,text,name,class,label_complete
0,0,Heavenly Sentiments Spray Peaceful White Lilie...,name,Product,nameProduct
1,1,"{'pricecurrency': 'USD', 'lowprice': '202.95',...",offers,Product,offersProduct
2,2,"{'reviewcount': '4', 'ratingvalue': '5.00'},{'...",aggregaterating,Product,aggregateratingProduct
3,3,"[{'author': 'from -', 'description': 'Arrangem...",review,Product,reviewProduct
4,4,NewCondition NewCondition NewCondition NewCond...,itemcondition,Product,itemconditionProduct
...,...,...,...,...,...
13231,13231,"{'name': 'Бахрах-М', 'address': 'Россия'},{'ad...",publisher,Book,publisherBook
13232,13232,"{'pricecurrency': 'RUB', 'availability': 'http...",offers,Book,offersBook
13233,13233,"2011.0,2016.0,2006.0,2018.0,2016.0,2020.0,2013...",datepublished,Book,datepublishedBook
13234,13234,736 140 560 208 208 32 288 576 272 264 286 2...,numberofpages,Book,numberofpagesBook


In [16]:
test = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/test_tables_seq_9000.csv')
test.drop(test[(test['name']=='inalbum') &(test['class']=='MusicRecording')].index, inplace=True)
test.drop(test[(test['name']=='performer') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventattendancemode') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='eventstatus') &(test['class']=='Event')].index, inplace=True)
test.drop(test[(test['name']=='owns') &(test['class']=='LocalBusiness')].index, inplace=True)
test["label_complete"] = test["name"] + test["class"]
num_columns = len(set.union(set(test.label_complete.unique())))
test

,Unnamed: 0,text,name,class,label_complete
0,0,Plauen | Indonesien Freiburg | Iraq-Kurdistan ...,name,Event,nameEvent
1,1,2022-03-27T16:30:00+01:00 2020-11-15T22:00:00+...,enddate,Event,enddateEvent
2,2,"{'name': 'Abenteuer Arena'},{'name': 'Paulussa...",location,Event,locationEvent
3,3,"{'name': 'Freiträumer Festival'},{'name': 'MUN...",organizer,Event,organizerEvent
4,4,Máy làm kem Sunny Way RB1119A TỦ MÁT 2 CÁNH KÍ...,name,Product,nameProduct
...,...,...,...,...,...
48530,48530,"{'name': 'Virtually'},{'name': 'Virtually'},{'...",location,Event,locationEvent
48531,48531,{'email': 'c&#104;i&#108;&#100;&#114;&#101;n&#...,organizer,Event,organizerEvent
48532,48532,Marina Olivier Celia Tatjana Valentine Gaia Ré...,givenname,Person,givennamePerson
48533,48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,familyname,Person,familynamePerson


In [17]:
test.drop(['Unnamed: 0','class'], axis=1, inplace=True)
test['text'] = test.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [18]:
small.drop(['Unnamed: 0','class'], axis=1, inplace=True)
small['text'] = small.text.apply(lambda x: re.sub("[^0-9a-zA-Z-@]+", " ", x))

In [22]:
test['label'] = test.label_complete.map(labels_new)

test.drop(['label_complete', 'name'], axis=1, inplace=True)
test

,text,label
0,Plauen Indonesien Freiburg Iraq-Kurdistan Boch...,130.0
1,2022-03-27T16 30 00 01 00 2020-11-15T22 00 00 ...,68.0
2,name Abenteuer Arena name Paulussaal name Deu...,115.0
3,name Freitr umer Festival name MUNDOlogia nam...,153.0
4,M y l m kem Sunny Way RB1119A T M T 2 C NH K N...,136.0
...,...,...
48530,name Virtually name Virtually name Virtually ...,115.0
48531,email c 104 i 108 100 114 101 n 115 st 97 f 1...,153.0
48532,Marina Olivier Celia Tatjana Valentine Gaia R ...,83.0
48533,Shunina de LAGARDE van der Beek Mavrenko BOITE...,71.0


In [23]:
small['label'] = small.label_complete.map(labels_new)

small.drop(['label_complete', 'name'], axis=1, inplace=True)
small

,text,label
0,Heavenly Sentiments Spray Peaceful White Lilie...,136.0
1,pricecurrency USD lowprice 202 95 highprice 2...,147.0
2,reviewcount 4 ratingvalue 5 00 reviewcount 25...,16.0
3,author from - description Arrangement was mad...,177.0
4,NewCondition NewCondition NewCondition NewCond...,108.0
...,...,...
13231,name - address address name name - address na...,166.0
13232,pricecurrency RUB availability http schema or...,145.0
13233,2011 0 2016 0 2006 0 2018 0 2016 0 2020 0 2013...,54.0
13234,736 140 560 208 208 32 288 576 272 264 286 229...,141.0


In [24]:
test = test[~test['label'].isna()]

In [25]:
small = small[~small['label'].isna()]

In [26]:
test['label'] = test.label.astype(int)

/tmp/ipykernel_1526305/2755059116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'] = test.label.astype(int)


In [27]:
small['label'] = small.label.astype(int)

/tmp/ipykernel_1526305/1106728590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['label'] = small.label.astype(int)


In [28]:
len (test['label'].unique())

199

In [29]:
len (small['label'].unique())

199

In [30]:
small_list = small['label'].unique()

In [31]:
test_list = test['label'].unique()

In [32]:
set(test_list) - set(small_list)

set()

In [33]:
set(small_list) - set(test_list)

set()

In [34]:
sorted(small.label.unique()) ==sorted(test.label.unique())

True

In [35]:
test.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_tables_prepped_small.csv', index=False)

In [36]:
small.to_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/small_train_tables_prepped.csv', index=False)

# Eval small

In [7]:
from sklearn.metrics import f1_score
import pandas as pd
from matplotlib.pyplot import figure

In [8]:
test_small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/test_tables_prepped_small.csv')

In [9]:
pred_small = pd.read_csv('../../../src/data/schemafiltereddata/TrainTestTables/tinybert/predict_small.txt', sep='\t')

In [11]:
f1_score(pred_small.prediction, test_small.label, average='micro')

0.6908873593537485